In [ ]:
import pandas as pd
import json
import json, pprint, re, datetime
import mwparserfromhell as mw
import re
import calendar
from daterangeparser import parse
%load_ext autoreload
%autoreload 2
import processing # this makes the "processing" folder available as a python module
from features.date import get_daterange

In [ ]:
data= "../datasets/battle-fields-0.json"
df= pd.read_json(data)
df['start_day']='empty'
df['start_month']='empty'
df['start_year']='empty'
df['end_day']='empty'
df['end_month']='empty'
df['end_year']='empty'

In [ ]:
data= "../datasets/battle-fields-0.json"
battles = json.load(open(data))

dates = [b["infobox"]["date"] for b in battles if b["infobox"].get("date") != None]
wi = mw.parse(dates[3])

parsed_dates = [get_daterange(d) for d in dates[:100]]

In [ ]:
df = pd.DataFrame(parsed_dates)
df

In [ ]:

for i in range(5406,len(df['start_day'])):
    if 'error' in df.infobox[i]:
        df.start_day[i]='noinfo'
        df.end_day[i]='noinfo'
        df.start_month[i]='noinfo'
        df.end_month[i]='noinfo'
        df.start_year[i]='noinfo'
        df.end_year[i]='noinfo'
    elif 'date' not in df.infobox[i].keys():
        df.start_day[i]='nodate'
        df.end_day[i]='nodate'
        df.start_month[i]='nodate'
        df.end_month[i]='nodate'
        df.start_year[i]='nodate'
        df.end_year[i]='nodate'
    elif re.search('\sBC\s',df.infobox[i]['date']):
        if re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})\sBC',df.infobox[i]['date']):
            df.start_day[i]=int(re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})\sBC',df.infobox[i]['date']).group(1))
            month=re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})\sBC',df.infobox[i]['date']).group(2)
            df.start_month[i]=datetime.datetime.strptime(month[:3], '%b').month
            df.start_year[i]=-int(re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})\sBC',df.infobox[i]['date']).group(3))
            df.end_day[i]=df.start_day[i]
            df.end_month[i]=df.start_month[i]
            df.end_year[i]=df.start_year[i]
        if re.search('(\w{3,9}),\s(\d{1,4})\sBC',df.infobox[i]['date']):
            df.start_day[i]=0
            month=re.search('(\w{3,9}),\s(\d{1,4})\sBC',df.infobox[i]['date']).group(1)
            df.start_month[i]=datetime.datetime.strptime(month[:3], '%b').month
            df.start_year[i]=-int(re.search('(\w{3,9}),\s(\d{1,4})\sBC',df.infobox[i]['date']).group(2))
            df.end_day[i]=df.start_day[i]
            df.end_month[i]=df.start_month[i]
            df.end_year[i]=df.start_year[i]
        elif re.search('(\w{3,9})\s(\d{1,2})\sand\s\d{1,2},\s(\d{1,4})\sBC',df.infobox[i]['date']):
            df.start_day[i]=int(re.search('(\w{3,9})\s(\d{1,2})\sand\s\d{1,2},\s(\d{1,4})\sBC',df.infobox[i]['date']).group(2))
            stmonth=re.search('(\w{3,9})\s(\d{1,2})\sand\s\d{1,2},\s(\d{1,4})\sBC',df.infobox[i]['date']).group(1)
            df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
            df.start_year[i]=-int(re.search('(\w{3,9})\s(\d{1,2})\sand\s\d{1,2},\s(\d{1,4})\sBC',df.infobox[i]['date']).group(3))
            df.end_day[i]=df.start_day[i]
            enmonth=stmonth
            df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
            df.end_year[i]=df.start_year[i]
        elif re.search('^(\w{3,9})/(\w{3,9})\s.*,\s(\d{1,4})\sBC',df.infobox[i]['date']):
            df.start_day[i]=0
            stmonth=re.search('(\w{3,9})/(\w{3,9})\s.*,\s(\d{1,4})\sBC',df.infobox[i]['date']).group(1)
            df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
            df.start_year[i]=-int(re.search('(\w{3,9})/(\w{3,9})\s.*,\s(\d{1,4})\sBC',df.infobox[i]['date']).group(3))
            df.end_day[i]=0
            enmonth=re.search('(\w{3,9})/(\w{3,9})\s.*,\s(\d{1,4})\sBC',df.infobox[i]['date']).group(2)
            df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
            df.end_year[i]=df.start_year[i]
        elif re.search('(\d{1,4})\sBC',df.infobox[i]['date']):
            df.start_day[i]=0
            df.start_month[i]=0
            df.start_year[i]=-int(re.search('(\d{1,4})\sBC',df.infobox[i]['date']).group(1))
            df.end_day[i]=0
            df.end_month[i]=0
            df.end_year[i]=df.start_year[i]  
            
    elif re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=int(re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(1))
        stmonth=re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(2)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(3))
        df.end_day[i]=int(re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(4))
        enmonth=re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(5)
        df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
        df.end_year[i]=int(re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(6))
    elif re.search('(\d{1,2})\s(\w{3,9})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=int(re.search('(\d{1,2})\s(\w{3,9})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(1))
        stmonth=re.search('(\d{1,2})\s(\w{3,9})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(2)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('(\d{1,2})\s(\w{3,9})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(5))
        df.end_day[i]=int(re.search('(\d{1,2})\s(\w{3,9})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(3))
        enmonth=re.search('(\d{1,2})\s(\w{3,9})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(4)
        df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
        df.end_year[i]=int(re.search('(\d{1,2})\s(\w{3,9})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(5))
    elif re.search('(\d{1,2})–(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=int(re.search('(\d{1,2})–(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(1))
        stmonth=re.search('(\d{1,2})–(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(3)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('(\d{1,2})–(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(4))
        df.end_day[i]=int(re.search('(\d{1,2})–(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(2))
        enmonth=re.search('(\d{1,2})–(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(3)
        df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
        df.end_year[i]=int(re.search('(\d{1,2})–(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(4))
    elif re.search('late\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=0
        df.start_month[i]=0
        df.start_year[i]=int(re.search('late\s(\d{1,4})',df.infobox[i]['date']).group(1))
        df.end_day[i]=df.start_day[i]
        df.end_month[i]=0
        df.end_year[i]=df.start_year[i]
    elif re.search('(\w{3,9})\s(\d{1,2})–(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=int(re.search('(\w{3,9})\s(\d{1,2})–(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(2))
        stmonth=re.search('(\w{3,9})\s(\d{1,2})–(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(1)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('(\w{3,9})\s(\d{1,2})–(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(4))
        df.end_day[i]=int(re.search('(\w{3,9})\s(\d{1,2})–(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(3))
        enmonth=stmonth
        df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
        df.end_year[i]=int(re.search('(\w{3,9})\s(\d{1,2})–(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(4))
    elif re.search('(\w{3,9})\s(\d{1,2})\s–\s(\w{3,9})\s(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=int(re.search('(\w{3,9})\s(\d{1,2})\s–\s(\w{3,9})\s(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(2))
        stmonth=re.search('(\w{3,9})\s(\d{1,2})\s–\s(\w{3,9})\s(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(1)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('(\w{3,9})\s(\d{1,2})\s–\s(\w{3,9})\s(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(5))
        df.end_day[i]=int(re.search('(\w{3,9})\s(\d{1,2})\s–\s(\w{3,9})\s(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(4))
        enmonth=re.search('(\w{3,9})\s(\d{1,2})\s–\s(\w{3,9})\s(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(3)
        df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
        df.end_year[i]=int(re.search('(\w{3,9})\s(\d{1,2})\s–\s(\w{3,9})\s(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(5))
    elif re.search('(\w{3,9})\s(\d{1,4})\s–\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=0
        stmonth=re.search('(\w{3,9})\s(\d{1,4})\s–\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(1)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('(\w{3,9})\s(\d{1,4})\s–\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(2))
        df.end_day[i]=0
        enmonth=re.search('(\w{3,9})\s(\d{1,4})\s–\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(3)
        df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
        df.end_year[i]=int(re.search('(\w{3,9})\s(\d{1,4})\s–\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(4))
    elif re.search('(\d{1,2})\sand\s\d{1,2}\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=int(re.search('(\d{1,2})\sand\s\d{1,2}\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(1))
        stmonth=re.search('(\d{1,2})\sand\s\d{1,2}\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(2)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('(\d{1,2})\sand\s\d{1,2}\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(3))
        df.end_day[i]=int(re.search('(\d{1,2})\sand\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(2))
        enmonth=stmonth
        df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
        df.end_year[i]=df.start_year[i]
    elif re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=int(re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(1))
        stmonth=re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(2)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[i]['date']).group(3))
        df.end_day[i]=df.start_day[i]
        enmonth=stmonth
        df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
        df.end_year[i]=df.start_year[i]
    elif re.search('\s(\w{3,9})\s(\d{1,2}),\s(\d{1,4})\sAD',df.infobox[i]['date']):
        df.start_day[i]=int(re.search('\s(\w{3,9})\s(\d{1,2}),\s(\d{1,4})\sAD',df.infobox[i]['date']).group(2))
        stmonth=re.search('\s(\w{3,9})\s(\d{1,2}),\s(\d{1,4})\sAD',df.infobox[i]['date']).group(1)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('\s(\w{3,9})\s(\d{1,2}),\s(\d{1,4})\sAD',df.infobox[i]['date']).group(3))
        df.end_day[i]=df.start_day[i]
        enmonth=stmonth
        df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
        df.end_year[i]=df.start_year[i]  
    elif re.search('(Spring|Summer|Winter|Autumn),\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=0
        df.start_month[i]=0
        df.start_year[i]=int(re.search('(Spring|Summer|Winter|Autumn),\s(\d{1,4})',df.infobox[i]['date']).group(2))
        df.end_day[i]=df.start_day[i]
        df.end_month[i]=0
        df.end_year[i]=df.start_year[i] 
    elif re.search('(\w{3,9})\s(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=int(re.search('(\w{3,9})\s(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(2))
        stmonth=re.search('(\w{3,9})\s(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(1)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('(\w{3,9})\s(\d{1,2}),\s(\d{1,4})',df.infobox[i]['date']).group(3))
        df.end_day[i]=df.start_day[i]
        enmonth=stmonth
        df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
        df.end_year[i]=df.start_year[i]    
    elif re.search('(\w{3,9})\sor\s(\w{3,9}),\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=0
        stmonth=re.search('(\w{3,9})\sor\s(\w{3,9}),\s(\d{1,4})',df.infobox[i]['date']).group(1)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('(\w{3,9})\sor\s(\w{3,9}),\s(\d{1,4})',df.infobox[i]['date']).group(3))
        df.end_day[i]=df.start_day[i]
        enmonth=stmonth
        df.end_month[i]=datetime.datetime.strptime(enmonth[:3], '%b').month
        df.end_year[i]=df.start_year[i]
    elif re.search('(\d{1,4})\sor\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=0
        df.start_month[i]=0
        df.start_year[i]=int(re.search('(\d{1,4})\sor\s(\d{1,4})',df.infobox[i]['date']).group(1))
        df.end_day[i]=df.start_day[i]
        df.end_month[i]=0
        df.end_year[i]=df.start_year[i]
    elif re.search('^(\w{3,9}})\s(\d{1,4})',df.infobox[i]['date']):
        df.start_day[i]=0
        stmonth=re.search('^(\w{3,9}})\s(\d{1,4})',df.infobox[i]['date']).group(1)
        df.start_month[i]=datetime.datetime.strptime(stmonth[:3], '%b').month
        df.start_year[i]=int(re.search('^(\w{3,9}})\s(\d{1,4})',df.infobox[i]['date']).group(2))
        df.end_day[i]=df.start_day[i]
        df.end_month[i]=df.start_month[i]
        df.end_year[i]=df.start_year[i]
    else:
        df.start_day[i]='emptyTreated'

    
        

            
        

In [ ]:
df.start_day[4845]=0
df.start_year

In [ ]:
re.search("(\d)+\s(.*)(\d{4})<",df.infobox[7]['date']).group(3)

In [ ]:
re.search('(\d{1,2})\s(\w{3,9})\s–\s(\d{1,2})\s(\w{3,9})\s(\d{1,4})',df.infobox[33]['date']).group(1)


In [ ]:
a=-int(re.search('\w{3,9}\sor\s\w{3,9},\s(\d{1,4})','July or August, 251').group(1))


In [ ]:
df.infobox[3822].keys()

In [ ]:
if 'date' not in df.infobox[3822].keys():
    print('NotIn')

In [ ]:
re.search('(\w{3,9})\s(\d{1,4})\s–\s(\w{3,9})\s(\d{1,4})',df.infobox[27250]['date']).group(0)

In [ ]:
df.start_year[5406]

In [ ]:
index=len(df['start_day'])-1
while index>0:
    if df['start_day'][index] != 'empty':
        print(index)
        break
    index-=1

In [ ]:
index=44
while index<len(df['start_day']):
    if df['start_day'][index] == 'empty':
        print(index)
        break
    index+=1

In [ ]:
df.infobox[5406]

In [ ]:
re.search('(Spring|Summer|Winter|Autumn),\s(\d{1,4})',df.infobox[5406]['date'])

In [ ]:
df['start_day'].values_count()

In [ ]:
df.start_year[5406]

In [ ]:
re.search('(\d{1,4})\sBC',df.infobox[3667]['date']).group(1)

In [ ]:
df.start_day[3858]

In [ ]:
df.loc['start_day'=='emptyTreated'].count()

In [ ]:
df['start_day'].value_counts()

In [ ]:
5407-2859-540

In [ ]:
start, end = parse(df.infobox[2]['date'])

In [ ]:
start

In [ ]:
df.infobox[2]['date']

In [ ]:
end